In [1]:
pip install lightgbm pandas scikit-learn joblib


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------------------------------ --- 1.3/1.5 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

In [3]:
 # Load dataset
df = pd.read_csv("Food_Delivery_Times.csv")

In [4]:
df.head()

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [5]:
# Drop 'Order_ID' and handle missing values
df.drop(columns=['Order_ID'], inplace=True)
df['Courier_Experience_yrs'].fillna(df['Courier_Experience_yrs'].median(), inplace=True)
for col in ['Weather', 'Traffic_Level', 'Time_of_Day']:
    df[col].fillna(df[col].mode()[0], inplace=True)

C:\Users\hvhar\AppData\Local\Temp\ipykernel_11456\662426898.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Courier_Experience_yrs'].fillna(df['Courier_Experience_yrs'].median(), inplace=True)
C:\Users\hvhar\AppData\Local\Temp\ipykernel_11456\662426898.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [6]:
df.head()

,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [7]:
# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, drop_first=True)

In [8]:
# Split features and target
X = df_encoded.drop('Delivery_Time_min', axis=1)
y = df_encoded['Delivery_Time_min']

In [9]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [11]:
# Train LightGBM model
model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=7)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 325
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 14
[LightGBM] [Info] Start training from score 57.053750
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMRegressor(max_depth=7)

In [12]:
# Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [13]:
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

RMSE: 9.66
R² Score: 0.79


In [14]:
# Save model and scaler
joblib.dump(model, "lgb_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [15]:


# Step 1: Manual input
sample_input = {
    'Distance_km': 5.0,
    'Weather': 'Rainy',
    'Traffic_Level': 'High',
    'Time_of_Day': 'Evening',
    'Vehicle_Type': 'Bike',
    'Preparation_Time_min': 20,
    'Courier_Experience_yrs': 2.5
}

# Step 2: Convert to DataFrame
input_df = pd.DataFrame([sample_input])

# Step 3: One-hot encode
input_encoded = pd.get_dummies(input_df)

# Step 4: Add missing columns
expected_cols = model.feature_name_
for col in expected_cols:
    if col not in input_encoded.columns:
        input_encoded[col] = 0

# Step 5: Reorder columns
input_encoded = input_encoded[expected_cols]

# ✅ Step 6: Scale without column names
input_scaled = scaler.transform(input_encoded.values)
# Step 7: Predict
predicted_time = model.predict(input_scaled)[0]
print(f"🚚 Predicted Delivery Time: {predicted_time:.2f} minutes")




🚚 Predicted Delivery Time: 23.12 minutes


c:\Users\hvhar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
